<a href="https://colab.research.google.com/github/SeohyunLee0321/GDSC/blob/main/%ED%98%BC%EA%B3%B5%EB%A8%B8%EC%8B%A0_Chapter7_2_%EC%8B%AC%EC%B8%B5%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 패션 MNIST 데이터셋 불러오기
from tensorflow import keras
(train_input, train_target), (test_input, test_target) =\
  keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
# 픽셀값 0~255 에서 0~1로 변환
# 28 * 28 크기의 2차원 배열 -> 784 크기의 1차원 배열로 펼치기
# train_test_split() 함수로 훈련 세트와 검증 세트로 나누기
from sklearn.model_selection import train_test_split
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size = 0.2, random_state = 42
)

In [ ]:
# 시그모이드 활성화 함수를 사용한 은닉층과 소프트맥스 함수를 사용한 출력층을 케라스의 Dense 클래스로 만들기
# 케사스에서 신경망의 첫 번째 층은 input_shape 매개변수로 입력의 크기 지정 꼭 필요
dense1 = keras.layers.Dense(100, activation = 'sigmoid', input_shape = (784,))
dense2 = keras.layers.Dense(10, activation = 'softmax')

**심층 신경망 만들기**

In [ ]:
# dense1, dense2 객체를 Sequential 클래스에 추가하여 심층 신경망 만들기
model = keras.Sequential([dense1, dense2])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


**층을 추가하는 다른 방법**

In [ ]:
# dense1, dense2 객체는 따로 저장하여 쓸 일이 없기 때문에 Sequential 클래스의 생성자 안에서 바로 Dense 클래스의 객체를 만드는 경우가 많음
model = keras.Sequential([
    keras.layers.Dense(100, activation = 'sigmoid', input_shape = (784,),
                       name = 'hidden'),
    keras.layers.Dense(10, activation = 'softmax', name = 'output')
], name = '패션 MNIST 모델')

# 추가되는 층을 한눈에 쉽게 알아볼 수 있음
# 모델과 달리 층의 이름은 반드시 영문이어야 함

In [ ]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 아주 많은 층을 추가하거나, 조건에 따라 층을 추가하는 방법 add()
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation = 'sigmoid', input_shape = (784,)))
model.add(keras.layers.Dense(10, activation = 'softmax'))

# 한눈에 추가되는 층을 볼 수 있고, 프로그램 실행 시 동적으로 층을 선택하여 추가 가능

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 모델 훈련
model.compile(loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 6s 3ms/step - loss: 0.5628 - accuracy: 0.8097
Epoch 2/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.4091 - accuracy: 0.8530
Epoch 3/5
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3739 - accuracy: 0.8668
Epoch 4/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3519 - accuracy: 0.8728
Epoch 5/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3354 - accuracy: 0.8777


**렐루 함수**

**시그모이드 함수의 단점** : 함수의 오른쪽과 왼쪽 끝으로 갈수록 그래프가 누워있기 때문에 올바른 출력을 만드는데 신속하게 대응하지 못함

층이 많은 심층 신경망일수록 효과가 누적되어 학습이 더 힘듦

**렐루 함수** : 입력이 양수일 경우 활성화 함수가 없는 것처럼 입력을 통과시키고 음수일 경우에는 0으로 만듦

max(0, z)와 같이 쓸 수 있음 -> z가 0보다 크면 z를 출력하고 z가 0보다 작으면 0을 출력

In [ ]:
# 패션 MNIST 데이터를 1차원으로 펼치기 위해 reshape() 메서드 사용해옴
# 케라스의 Flatten 층을 사용하는 것도 좋음
# Flatten 클래스 : 배치 차원을 제외하고 나머지 입력 차원을 모두 일렬로 펼치는 역할만 함
# 입력에 곱해지는 가중치나 절편이 없음 -> 인공 신경망의 성능을 위해 기여하는 바는 없음
# 입력층과 은닉층 사이에 Flatten 층을 추가하기 때문에 '층'이라고 부름
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape = (28, 28)))
model.add(keras.layers.Dense(100, activation = 'relu'))
model.add(keras.layers.Dense(10, activation = 'softmax'))

# 첫 번째 Dense 층에 있던 input_shape 매개변수를 Flatten 층으로 옮김, 첫 번째 Dense 층의 활성화 함수를 'relu'로 바꿈
# 신경망의 깊이가 3인 것은 아님 -> Flatten 클래스는 학습하는 층이 아니기 때문

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


Flatten 클래스에 포함된 모델 파라미터는 0개

케라스의 Flatten 층을 신경망 모델에 추가하면 입력값의 차원을 짐작할 수 있는 것이 장점

784개의 입력이 첫 번째 은닉층에 전달된다는 것을 알 수 있음

In [ ]:
# 훈련 데이터를 다시 준비하여 모델 훈련, reshape() 사용하지 않음
(train_input, train_target), (test_input, test_target) =\
  keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size = 0.2, random_state = 42
)

In [ ]:
# 모델 컴파일 후 훈련
model.compile(loss ='sparse_categorical_crossentropy', metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs = 5)

# 시그모이드 함수를 사용했을 때와 비교하여 성능 향상

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5287 - accuracy: 0.8142
Epoch 2/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3931 - accuracy: 0.8599
Epoch 3/5
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3530 - accuracy: 0.8744
Epoch 4/5
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3353 - accuracy: 0.8791
Epoch 5/5
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3180 - accuracy: 0.8861


In [ ]:
# 검증 세트에서의 성능 확인
model.evaluate(val_scaled, val_target)

# 1절의 은닉층을 추가하지 않은 경우보다 성능 향상

375/375 [==============================] - 1s 2ms/step - loss: 0.3803 - accuracy: 0.8770


[0.38027507066726685, 0.8769999742507935]

**옵티마이저**

옵티마이저 : 케라스에서 제공하는 다양한 종류의 경사 하강법 알고리즘

In [ ]:
# 가장 기본적인 옵티마이저는 확률적 경사 하강법인 SGD
# compile() 메서드의 optimizer 매개변수를 'sgd'로 지정
model.compile(optimizer = 'sgd', loss = 'sparse_categorical_crossentropy',
              metrics = 'accuracy')

# 위와 아래의 코드는 동일
# sgd = keras.optimizers.SGD()
# model.compile(optimizer = sgd, loss = 'sparse_categorical_crossentropy',
#               metrics = 'accuracy')

# 원래 sgd = keras.optimizers.SGD() 처럼 SGD 클래스 객체를 만들어 사용해야하지만 'sgd'라 지정하면 자동으로 SGD 클래스 객체를 만들어주는 것

In [ ]:
# SGD 클래스의 학습률 기본값이 0.01 -> 바꾸고 싶다면 원하는 학습률을 learning_rate 매개변수에 지정하여 사용
sgd = keras.optimizers.SGD(learning_rate = 0.1)

SGD 클래스의 momentum 매개변수의 기본값은 0

0보다 큰 값으로 지정하면 이전의 그래디언트를 가속도처럼 사용하는 **모멘텀 최적화** 사용

보통 momentum 매개변수는 0.9 이상을 지정

In [ ]:
# SGD 클래스의 nesterov 매개변수를 기본값 False 에서 True로 바꾸면 네스테로프 모멘텀 최적화 (네스테로프 가속 경사) 사용
sgd = keras.optimizers.SGD(momentum = 0.9, nesterov = True)

# 네스테로프 모멘텀은 모멘텀 최적화를 2번 반복하여 구현 -> 기본 확률적 경사 하강법보다 더 나은 성능 제공 (보통)

**적응적 학습률** : 모델이 최적점 가까이 갈수록 학습률을 낮추면 안정적으로 최적점에 수렴할 가능성이 높음

-> 학습률 매개변수를 튜닝하는 수고를 덜 수 있음

적응적 학습률을 사용하는 대표적인 옵티마이저는 Adagrad와 RMSprop

In [ ]:
# 모델 재생성
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape = (28, 28)))
model.add(keras.layers.Dense(100, activation = 'relu'))
model.add(keras.layers.Dense(10, activation = 'softmax'))

adagrad = keras.optimizers.Adagrad()
model.compile(optimizer = adagrad, loss = 'sparse_categorical_crossentropy',
              metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 4s 3ms/step - loss: 1.1469 - accuracy: 0.6667
Epoch 2/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.7665 - accuracy: 0.7559
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.6821 - accuracy: 0.7792
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.6379 - accuracy: 0.7917
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.6089 - accuracy: 0.8009


In [ ]:
# 모델 재생성
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape = (28, 28)))
model.add(keras.layers.Dense(100, activation = 'relu'))
model.add(keras.layers.Dense(10, activation = 'softmax'))

rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer = rmsprop, loss = 'sparse_categorical_crossentropy',
              metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5343 - accuracy: 0.8142
Epoch 2/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3920 - accuracy: 0.8585
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3530 - accuracy: 0.8739
Epoch 4/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3350 - accuracy: 0.8797
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3206 - accuracy: 0.8845


In [ ]:
# Adam 클래스의 매개변수 기본값을 사용하여 패션 MNIST 모델 훈련
# 모델 재생성
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape = (28, 28)))
model.add(keras.layers.Dense(100, activation = 'relu'))
model.add(keras.layers.Dense(10, activation = 'softmax'))

In [ ]:
# compile() 메서드의 optimizer 를 'adam'으로 설정하고 5번의 에포크 동안 훈련
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',
              metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs = 5)

# RMSprop 사용했을 때와 거의 같은 성능 보임

Epoch 1/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5322 - accuracy: 0.8155
Epoch 2/5
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3934 - accuracy: 0.8597
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3489 - accuracy: 0.8736
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3298 - accuracy: 0.8809
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3094 - accuracy: 0.8864


In [ ]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3811 - accuracy: 0.8608


[0.38106444478034973, 0.8607500195503235]